In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install openpyxl

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result_cv/"

In [ ]:
file_train = "Train_M.musculus_succ.csv"

In [ ]:
# Build an amino acid dictionary
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
#  Load train data and pre-processing
df_train =pd.read_csv(path_data+file_train, delimiter= ',')
texts_train =[]
for i in df_train['Peptide_WS_33']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])
Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
#one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape


(1242, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
# KD2_succi
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers, losses

# Teacher 1
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model

# Teacher 2
def create_bilstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# Student
# Student
def create_student():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
#  KD2_succi: Knowlege distillation
import tensorflow as tf
from tensorflow.keras import optimizers

class Distiller(tf.keras.Model):
    def __init__(self, teacher1, teacher2, student, alpha=0.5, temperature=3.0):
        super(Distiller, self).__init__()
        self.teacher1 = teacher1
        self.teacher2 = teacher2
        self.student = student
        self.alpha = alpha
        self.temperature = temperature

        # Optimizer for the student
        self.student_optimizer = AdamW(learning_rate=0.0001)

    def compile(self, metrics, student_loss_fn, distillation_loss_fn):
        super(Distiller, self).compile(metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def call(self, inputs):
        """
        This method defines how the model processes inputs.
        Args:
        - inputs: Input data (Xtest_student or any other data).

        Returns:
        - Output logits from the student model.
        """
        student_logits = self.student(inputs, training=False)
        return student_logits

    def train_step(self, data):
        x, y = data

        # Sequentially forward pass through teachers
        teacher1_logits = self.teacher1(x, training=False)
        teacher2_logits = self.teacher2(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass through student
            student_logits = self.student(x, training=True)

            # Compute the task loss (for hard labels)
            student_loss = self.student_loss_fn(y, student_logits)

            # Compute distillation losses (for soft labels)
            distill_loss1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher1_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            distill_loss2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher2_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            # Combined loss
            total_loss = self.alpha * student_loss + (1 - self.alpha) * (distill_loss1 + distill_loss2)

        # Compute gradients and update student model
        grads = tape.gradient(total_loss, self.student.trainable_variables)
        self.student_optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        # Return results
        results = {
            'student_loss': student_loss,
            'distill_loss1': distill_loss1,
            'distill_loss2': distill_loss2
        }
        results.update({m.name: m.result() for m in self.metrics})
        return results

    def test_step(self, data):
        x, y = data
        student_logits = self.student(x, training=False)

        # Compute the task loss (for hard labels)
        student_loss = self.student_loss_fn(y, student_logits)

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        return {
            'student_loss': student_loss,
            **{m.name: m.result() for m in self.metrics}
        }



In [ ]:
# Transfer learning from Generic dl set to species
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras import models
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    student_fold = create_student()
    teacher1_fold = models.load_model(path_model +"model_teacher21.h5") # Sử dụng teacher giống nhau trên cùng loài
    teacher2_fold = models.load_model(path_model +"model_teacher22.h5")
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,

    )
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)
    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

#  Show TP, TN, FP, FN
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_M.musclus_KD2_Succi.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_M.musclus_KD2_Succi.xlsx'")

Training on fold 1/10...


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 27s 129ms/step - AUC: 0.6124 - categorical_accuracy: 0.6008 - distill_loss1: 0.0669 - distill_loss2: 0.1220 - loss: 0.5000 - student_loss: 0.9231 - val_AUC: 0.6683 - val_categorical_accuracy: 0.6160 - val_loss: 0.5000 - val_student_loss: 0.6879
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - AUC: 0.6443 - categorical_accuracy: 0.5976 - distill_loss1: 0.0586 - distill_loss2: 0.1065 - loss: 0.5000 - student_loss: 0.7863 - val_AUC: 0.6594 - val_categorical_accuracy: 0.6080 - val_loss: 0.5000 - val_student_loss: 0.6881
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 11s 89ms/step - AUC: 0.6589 - categorical_accuracy: 0.6101 - distill_loss1: 0.0510 - distill_loss2: 0.0988 - loss: 0.5000 - student_loss: 0.7281 - val_AUC: 0.7090 - val_categorical_accuracy: 0.6240 - val_loss: 0.5000 - val_student_loss: 0.6692
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 12s 110ms/step - AUC: 0.7182 - categorical_accuracy: 0.6563 - distill_loss1: 0.0508 - distill_loss2: 0.0933 - loss: 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


...

70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 103ms/step - AUC: 0.5058 - categorical_accuracy: 0.5053 - distill_loss1: 0.0797 - distill_loss2: 0.1322 - loss: 0.5000 - student_loss: 1.0680 - val_AUC: 0.7577 - val_categorical_accuracy: 0.7339 - val_loss: 0.5000 - val_student_loss: 0.6497
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 8s 116ms/step - AUC: 0.6433 - categorical_accuracy: 0.6039 - distill_loss1: 0.0622 - distill_loss2: 0.1101 - loss: 0.5000 - student_loss: 0.8278 - val_AUC: 0.7589 - val_categorical_accuracy: 0.7258 - val_loss: 0.5000 - val_student_loss: 0.6554
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step - AUC: 0.6539 - categorical_accuracy: 0.6209 - distill_loss1: 0.0642 - distill_loss2: 0.1132 - loss: 0.5000 - student_loss: 0.8369 - val_AUC: 0.7447 - val_categorical_accuracy: 0.7258 - val_loss: 0.5000 - val_student_loss: 0.6372
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 10s 84ms/step - AUC: 0.6917 - categorical_accuracy: 0.6286 - distill_loss1: 0.0544 - distill_loss2: 0.0951 - loss: 0.5000 - stu

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


70/70 ━━━━━━━━━━━━━━━━━━━━ 25s 134ms/step - AUC: 0.5700 - categorical_accuracy: 0.5611 - distill_loss1: 0.0688 - distill_loss2: 0.1226 - loss: 0.5000 - student_loss: 0.9354 - val_AUC: 0.6937 - val_categorical_accuracy: 0.6613 - val_loss: 0.5000 - val_student_loss: 0.6687
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 102ms/step - AUC: 0.6352 - categorical_accuracy: 0.5890 - distill_loss1: 0.0616 - distill_loss2: 0.1099 - loss: 0.5000 - student_loss: 0.7985 - val_AUC: 0.7175 - val_categorical_accuracy: 0.6532 - val_loss: 0.5000 - val_student_loss: 0.6505
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 104ms/step - AUC: 0.6749 - categorical_accuracy: 0.6416 - distill_loss1: 0.0554 - distill_loss2: 0.1024 - loss: 0.5000 - student_loss: 0.7464 - val_AUC: 0.7238 - val_categorical_accuracy: 0.6613 - val_loss: 0.5000 - val_student_loss: 0.6266
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 96ms/step - AUC: 0.7253 - categorical_accuracy: 0.6677 - distill_loss1: 0.0514 - distill_loss2: 0.0922 - loss: 0.5000 - stu

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW

k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    teacher1_fold = create_cnn_model()
    teacher2_fold = create_bilstm_model()
    student_fold = create_student()
    teacher1_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher2_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher1_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    teacher2_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    # Re-initialize the distiller for each fold
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

     # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,
        verbose=1
    )

    # Save result of each foldd
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

# Show TP, TN, FP, FN for each fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into the file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_M.musculus_KD2_Succ1.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_M.musculus_KD2_Succ1.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - AUC: 0.5359 - categorical_accuracy: 0.5273 - loss: 0.8205
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6262 - categorical_accuracy: 0.5912 - loss: 0.7396
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6436 - categorical_accuracy: 0.6215 - loss: 0.7194
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - AUC: 0.6313 - categorical_accuracy: 0.6205 - loss: 0.7295
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6604 - categorical_accuracy: 0.6417 - loss: 0.7142
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6495 - categorical_accuracy: 0.6105 - loss: 0.6896
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6852 - categorical_accuracy: 0.6369 - loss: 0.6621
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6835 - categorical_accuracy: 0.6310 - loss: 0.6557
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6737 - categorical_accuracy: 0.6356 - loss: 0.6652
Epoch 10/50
70/70 ━━━

...

70/70 ━━━━━━━━━━━━━━━━━━━━ 12s 45ms/step - AUC: 0.5924 - categorical_accuracy: 0.5618 - distill_loss1: 0.0622 - distill_loss2: 0.1415 - loss: 0.5000 - student_loss: 0.9203 - val_AUC: 0.6944 - val_categorical_accuracy: 0.6613 - val_loss: 0.5000 - val_student_loss: 0.6909
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - AUC: 0.6832 - categorical_accuracy: 0.6468 - distill_loss1: 0.0487 - distill_loss2: 0.1179 - loss: 0.5000 - student_loss: 0.7612 - val_AUC: 0.6611 - val_categorical_accuracy: 0.6290 - val_loss: 0.5000 - val_student_loss: 0.6908
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - AUC: 0.7158 - categorical_accuracy: 0.6761 - distill_loss1: 0.0437 - distill_loss2: 0.1112 - loss: 0.5000 - student_loss: 0.7109 - val_AUC: 0.7291 - val_categorical_accuracy: 0.6855 - val_loss: 0.5000 - val_student_loss: 0.6676
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - AUC: 0.7294 - categorical_accuracy: 0.6796 - distill_loss1: 0.0370 - distill_loss2: 0.1010 - loss: 0.5000 - studen

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - AUC: 0.3222 - categorical_accuracy: 0.3488 - loss: 1.8060
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5839 - categorical_accuracy: 0.5642 - loss: 0.7951
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6000 - categorical_accuracy: 0.5582 - loss: 0.7831
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6557 - categorical_accuracy: 0.6323 - loss: 0.7355
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6556 - categorical_accuracy: 0.6200 - loss: 0.7035
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.6220 - categorical_accuracy: 0.5901 - loss: 0.7352
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.7065 - categorical_accuracy: 0.6579 - loss: 0.6543
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.6371 - categorical_accuracy: 0.6149 - loss: 0.7061
Epoch 9/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - AUC: 0.6932 - categorical_accuracy: 0.6513 - loss: 0.6548
Epoch 10/50
70/70 ━━━

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


70/70 ━━━━━━━━━━━━━━━━━━━━ 14s 49ms/step - AUC: 0.5748 - categorical_accuracy: 0.5426 - distill_loss1: 0.0622 - distill_loss2: 0.1436 - loss: 0.5000 - student_loss: 0.9610 - val_AUC: 0.7133 - val_categorical_accuracy: 0.6613 - val_loss: 0.5000 - val_student_loss: 0.6437
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - AUC: 0.6566 - categorical_accuracy: 0.6251 - distill_loss1: 0.0512 - distill_loss2: 0.1289 - loss: 0.5000 - student_loss: 0.8711 - val_AUC: 0.7289 - val_categorical_accuracy: 0.6613 - val_loss: 0.5000 - val_student_loss: 0.6358
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - AUC: 0.7048 - categorical_accuracy: 0.6605 - distill_loss1: 0.0442 - distill_loss2: 0.1152 - loss: 0.5000 - student_loss: 0.7479 - val_AUC: 0.7609 - val_categorical_accuracy: 0.6855 - val_loss: 0.5000 - val_student_loss: 0.6034
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - AUC: 0.7137 - categorical_accuracy: 0.6620 - distill_loss1: 0.0383 - distill_loss2: 0.1094 - loss: 0.5000 - studen